# Phase 1: Basic Machine Translation with RNN

In this phase we'll be building a machine learning model for translation task, using PyTorch and TorchText. Translation will be done from German to English using a specific type of architecture called Sequence to Sequence Models. The same architecture is being used for various tasks like summarization, utterance generation, etc.

In this first phase, we'll start simple to understand the general concepts by implementing the model using RNN

## Introduction

The most common sequence-to-sequence (seq2seq) models are encoder-decoder models, which commonly use a recurrent neural network (RNN) to encode the source (input) sentence into a single vector. In this notebook, we'll refer to this single vector as a context vector. We can think of the context vector as being an abstract representation of the entire input sentence. This vector is then decoded by a second RNN which learns to output the target (output) sentence by generating it one word at a time.


![](https://drive.google.com/uc?id=1AZ8D6qSu_rMWOHa4STtLaPM4cKvTdZvL)

The source sentence "guten morgen" is passed through an embedding layer and then input into the EncoderRNN (green). At each time step $t$, the input to the EncoderRNN is both the embedding of the current word $x_t$, $e(x_t)$ and the hidden state from the previous time step, $h_{t-1}$. Then the EncoderRNN outputs a new hidden state $h_t$. This can be represented as following:

$$h_t = EncoderRNN(e(x_t), h_{t-1})$$


We also append the `<sos>` and `<eos>` tokens to indicate the start and end of the sentence.

Here, we will denote the source sentence as $X = \{x_1, x_2, ...., x_T\}$ where T is the number of words. We compute the hidden state for each time step as mentioned above, and we use the final hidden state $h_T$, as the context vector, i.e $h_T = z$. Intuitively the context vector represents the source sentence.

Now we have the context vector, $z$, we can use it to start the decoding. The DecoderRNN (blue) is also similar to similar to EncoderRNN, at each time step, it takes the input as the embedding of the current word $y_t$, $e^{'}(y_t)$, as well as the hidden state  from the previous time step, $s_{t-1}$. Then the DecoderRNN outputs a new hidden state $s_t$. This can be represented as:

$$s_t = DecoderRNN(e^{'}(y_t), s_{t-1})$$

**Note**: The hidden state for the initial step of decoding is context vector, i.e $s_0 = z = h_T$, i.e the decoder initial hidden state is the final encoder hidden state.

**Note**: The embedding layers are different for the Encoder and Decoder since the source and target languages are different. Though in diagram it is represented in the same color (yellow), we differentiate it in the equations as $e$ for source embedding and $e^{'}$ for target embedding layer.

In the decoder, we need to go from the hidden state $s_t$ to an acutal word $\hat{y}_t$. This will be done by passing the state $s_t$ through a linear layer which predicts the most probable word.

$$\hat{y}_t = softmax(f(s_t))$$

The decoding happens step-by-step, i.e words are generated one after another. We always use `<sos>` as the first input to the decoder for the first time step, i.e $y_1 = $ \<sos\>. But for subsequent inputs, $y_{t>1}$ we use the actual, ground truth as the input.

Since during the inference the ground truth is not available we will use the predicted word $\hat{y}_t$ as the input word to the next time step $t+1$. During inference we will keep generating the words until the model outputs an \<eos\> token or for a certain number of steps.

Once we have our predicted target sequence, $\hat{Y} = \{\hat{y}_1, \hat{y}_2, ..., \hat{y}_T\}$, we compare it against out actual target sentence, $Y = \{y_1, y_2, ..., y_T\}$, to calculate our loss. We then use this loss to update the parameters of the model.

 

## Imports

In [ ]:
import time
import math
import random
import spacy

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

In [ ]:
!python -m spacy download de

In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

### Tokenization

In [ ]:
def tokenize_de(text):
    """
    Tokenizes the German text from a string into a list of tokens
    """
    # TODO
    # Implement the tokenization of german sentence using spacy_de
    return tokens

In [ ]:
def tokenize_en(text):
    """
    Tokenizes the English text from a string into a list of tokens
    """
    # TODO
    # Implement the tokenization of english sentence using spacy_en
    return tokens

### Fields

Learn about Torchtext Fields here:

- [Youtube Tutorials](https://www.youtube.com/watch?v=KRgq4VnCr7I)
- [Blog](http://anie.me/On-Torchtext/)

In Machine Translation, the source language and target languages are different.

Field defines how the data should be processed.

Since we are tokenizing using spacy, we can pass our tokenizer method to the argument tokenizer

In order to indicate the starting and ending of a sentence, we can init_token as <sos> and eos_token as <eos>

In [ ]:
SRC = Field(
        tokenize=tokenize_de,
        init_token='<sos>',
        eos_token='<eos>',
        lower=True)

TRG = Field(
        tokenize=tokenize_en,
        init_token='<sos>',
        eos_token='<eos>',
        lower=True)

### Translation Dataset

We will be using Multi30K dataset. Torchtext provides support for multi open datasets. This is a dataset with ~31,000 parallel English, German and French sentences.

exts specifies which languages to use as the source and target (source goes first) and fields specifies which field to use for the source and target.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

In [ ]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

In [ ]:
print(vars(train_data.examples[0]))

### Vocabulary

Using the **min_freq** argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. Tokens that appear only once are converted into an **\<unk\>** (unknown) token.

It is important to note that our vocabulary should only be built from the training set and not the validation/test set.

In [ ]:
# TODO
# build vocabulary on SRC, TRG fields using training data and a min_freq of 2

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

### Iterators

The final step of preparing the data is to create the iterators. These can be iterated on to return a batch of data which will have a src attribute (the PyTorch tensors containing a batch of numericalized source sentences) and a trg attribute (the PyTorch tensors containing a batch of numericalized target sentences).

When we get a batch of examples using an iterator we need to make sure that all of the source sentences are padded to the same length, the same with the target sentences. Luckily, TorchText iterators handle this for us!

We use a BucketIterator instead of the standard Iterator as it creates batches in such a way that it minimizes the amount of padding in both the source and target sentences.

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

In [ ]:
# sample checking
temp = next(iter(train_iterator))
temp.src.shape, temp.trg.shape

### Encoder

The encoder we are using is a 2 layer RNN. The paper we are implementing uses a 4-layer LSTM, but in the interest of training time we cut this down to 2-layers.

Once the source sentence is passed through the RNN, the final step's hidden state in all the layers will be used for decoding.

This final hidden state is called context vector, which represents the encoding of the source sentence. This will be used as the initial hidden state for decoder.

> Note: In order for this to work, the number of layers in encoder and decoder must be same, if not then some extra strategy needs to be applied. For example, if the encoder is 1 layer and decoder is 2 layers means, then encoder final hidden state needs to replicated and sent as initial state to decoder



In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, dropout):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # TODO
        # Declare the embedding layer
        # self.embedding = 

        # TODO
        # Declare the encoder rnn
        # self.rnn = 

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, src):
        # TODO
        # comment the src shape

        embedded_input = self.embedding(src)
        embedded_input = self.dropout(embedded_input)
        # TODO
        # comment the embedded_input shape

        outputs, (hidden) = self.rnn(embedded_input)
        # TODO
        # comment the shapes of outputs and hidden

        return hidden

### Decoder

The decoder we are using is a 2 layer RNN like encoder. The Decoder class does a single step of decoding, i.e. it ouputs single token per time-step.

The initial hidden and cell states to the decoder are context vectors $z^1, z^2$, which are the final hidden and cell states of the encoder from the same layer.

As we are only decoding one token at a time, the input tokens will always have a sequence length of 1. We unsqueeze the input tokens to add a sentence length dimension of 1. Then it is passed through the RNN. There is an additional Linear layer, used to make the predictions from the top layer hidden state.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        # TODO
        # declare the embedding layer
        # self.embedding = 

        # TODO
        # declare the decoder rnn
        # self.rnn = 
        
        # TODO
        # declare the output prediction layer
        # self.out = 

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, dec_input, hidden):
        # TODO
        # comment the shapes of dec_input & hidden inputs

        dec_input = dec_input.unsqueeze(0)
        # TODO
        # comment why this is required

        dec_input_embedded = self.embedding(dec_input)
        dec_input_embedded = self.dropout(dec_input_embedded)
        # TODO
        # comment the shape of dec_input_embedded

        output, (hidden) = self.rnn(dec_input_embedded, (hidden))
        # TODO
        # comment the shapes of output, hidden

        # TODO
        # pass the output through the prediction layers and get the logits of tokens
        # prediction = 

        return prediction, hidden

### Sequence-to-Sequence Model


For the final part of the implemenetation, we'll implement the seq2seq model. This will handle:

- receiving the input/source sentence
- using the encoder to produce the context vectors
- using the decoder to produce the predicted output/target sentence

The forward method takes the source sentence, target sentence.

When decoding, at each time-step we will predict what the next token in the target sequence will be from the previous tokens decoded, $\hat{y}_{t+1}=f(s_t^L)$. We will use the actual ground-truth next token in the sequence as the input to the decoder during the next time-step.

The first input to the decoder is the start of sequence <sos> token. As our trg tensor already has the <sos> token appended we get our $y_1$ by slicing into it. We know how long our target sentences should be (max_len), so we loop that many times. The last token input into the decoder is the one before the <eos> token


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, src, trg):
        # TODO
        # comment the shapes of src and trg

        # TODO
        # get the values of the following from inputs
        # batch_size = 
        # trg_len = 
        # trg_vocab_size = 

        # outputs => to store the predictions at each decoding step
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)

        hidden = self.encoder(src)

        # TODO
        # initial input to decoder is <sos> which indicates to start the decoding process
        # since the first token is always <sos> take that as initial decoder input
        # decoder_input = 

        # Now let's run the decoding till the trg_len steps
        # skipping the first step as we have taken input as <sos>
        for step in range(1, trg_len):
            # decode step
            output, hidden = self.decoder(decoder_input, hidden)

            # save the output
            outputs[step] = output

            # TODO
            # get the input for decoder, for the next step
            # decoder_input = 
        
        return outputs

In [ ]:
INPUT_DIM = 
OUTPUT_DIM = 
ENC_EMB_DIM = 
DEC_EMB_DIM = 
HID_DIM = 
N_LAYERS = 2
DROPOUT = 0.5

In [ ]:
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DROPOUT)

model = Seq2Seq(encoder, decoder).to(device)

In [ ]:
def init_weights(model):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

### Optimizer

In [ ]:
optimizer = optim.Adam(model.parameters())

### Loss Criterion

In [ ]:
# TODO
# get the padding index from the target vocabulary
# TRG_PAD_IDX = 
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

### Training Method

At each iteration:

- get the source and target sentences from the batch, $X$ and $Y$
- zero the gradients calculated from the last batch
- feed the source and target into the model to get the output, $\hat{Y}$
- as the loss function only works on 2d inputs with 1d targets we need to flatten each of them with .view
- we slice off the first column of the output and target tensors (<sos> not used)
- calculate the gradients with loss.backward()
- clip the gradients to prevent them from exploding
- update the parameters of our model by doing an optimizer step
- update the loss

Finally, we return the loss that is averaged over all batches.

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        # TODO
        # comment the shapes of the src, trg

        # zero the gradients
        optimizer.step()

        # forward pass
        output = model(src, trg)        

        # TODO
        # reshape the output, trg to make it compatible for loss cal.
        loss = criterion(output, trg)

        # backward pass
        loss.backward()

        # gradient clipping
        nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update the parameters of the model
        optimizer.step()

        # update the loss
        epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

### Evaluation Method

In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            # TODO
            # comment the shapes of the src, trg

            # forward pass
            output = model(src, trg)        

            # TODO
            # reshape the output, trg to make it compatible for loss cal.
            loss = criterion(output, trg)

            # update the loss
            epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = elapsed_time - (elapsed_mins * 60)
    return elapsed_mins, elapsed_secs

### Training Loop

In [ ]:
N_EPOCHS = 10
CLIP = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iterator, criterion, optimizer, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # TODO
    # update the best validation loss and save the model 
    
    print(f"Epoch {epoch + 1} | Time: {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} |")
    print(f"\tValid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f} |")

### Testing

In [ ]:
# TODO
# load the trained model

# TODO
# calculate test loss by using test iterator
test_loss = 

print(f"\tTest Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |")